In [1]:
import pandas as pd 
import numpy as np 
import os 

In [2]:
essays_path = "../raw/llm-detect-ai-generated-text/train_essays.csv"
prompt_path = "../raw/llm-detect-ai-generated-text/train_prompts.csv"

daigt_path = "../raw/daigt/daigt_external_dataset.csv"

In [3]:
essays_df = pd.read_csv(essays_path)
prompt_df = pd.read_csv(prompt_path)

daigt_df = pd.read_csv(daigt_path)

In [4]:
daigt_df

,id,text,instructions,source_text
0,6060D28C05B6,Some schools in United States ofter classes fr...,\nTask: Write a persuasive essay on whether or...,\nWhen considering the pros and cons of attend...
1,60623DB5DE7A,"Four-day work week, a remarkable idea to conse...",\nTask: Research the advantages and disadvanta...,\nOne of the primary arguments for implementin...
2,607A39D981DE,Students and their families should consider an...,\nTask: \n\n1. Talk to your parents before tak...,\nBefore making any decisions about getting in...
3,60ACDFA1609E,Agree you will never grow if something beyond ...,\nTask: Write an essay discussing the benefits...,"\nRalph Waldo Emerson once said, ""Go confident..."
4,60AE13D3F07B,I think our character traits are formed by inf...,\nTask: Research and discuss how character tra...,\nHuman character traits are shaped by a wide ...
...,...,...,...,...
2416,F5FF5E9E553C,"On september 2, 2015 Generic_Name was entering...",\nTask: Research different kinds of medical pr...,\nBecoming a surgeon requires a great deal of ...
2417,F60545D8271E,I think that schools must have a after school ...,\nTask: Write an essay discussing why schools ...,\nSchools should offer an after school homewor...
2418,F610B3CBF3DF,"Winston Churchill once said ""success consists ...",\nTask: Write an essay about how having a few ...,\nIt’s human nature to be afraid to make mista...
2419,F610C7BCD9EC,Technology seems to be becoming more and more ...,\nTask: \n\nWrite an essay exploring the pros ...,\nOne of the main debates of 2020 for many stu...


In [8]:
daigt_tasks_only = daigt_df[daigt_df["instructions"].str.startswith("\nTask")]
daigt_tasks = daigt_df[['instructions', 'text']]
daigt_tasks = daigt_tasks.rename({'text': 'human-text'})
daigt_prompts = pd.DataFrame(daigt_tasks)

In [10]:
daigt_prompts

,instructions,text
0,\nTask: Write a persuasive essay on whether or...,Some schools in United States ofter classes fr...
1,\nTask: Research the advantages and disadvanta...,"Four-day work week, a remarkable idea to conse..."
2,\nTask: \n\n1. Talk to your parents before tak...,Students and their families should consider an...
3,\nTask: Write an essay discussing the benefits...,Agree you will never grow if something beyond ...
4,\nTask: Research and discuss how character tra...,I think our character traits are formed by inf...
...,...,...
2416,\nTask: Research different kinds of medical pr...,"On september 2, 2015 Generic_Name was entering..."
2417,\nTask: Write an essay discussing why schools ...,I think that schools must have a after school ...
2418,\nTask: Write an essay about how having a few ...,"Winston Churchill once said ""success consists ..."
2419,\nTask: \n\nWrite an essay exploring the pros ...,Technology seems to be becoming more and more ...


In [9]:
# daigt_prompts.to_csv("../prepared/train/daigt/daigt_prompts.csv", index=False)

## Testing with Mistral Instruct Model

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" # the device to load the model onto
#
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
def generate_essay(prompt):
    messages = [{
        "role": "user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    generated_ids = model.generate(
        model_inputs,
        max_new_tokens=7500,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

In [32]:
chosen_task = np.random.choice(daigt_tasks)
chosen_task

'\nTask: Research and analyze the benefits of enforcing curfews for teenagers who do not get into trouble, and what activities or resources can be implemented to ensure that they are staying out of trouble.'

In [36]:
prompt = f'''You are a student working on the following assignment.

Create an essay based on the following task in no more than a 100 words.
{chosen_task}
'''